In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.models import Model

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))
from deepexplain.tensorflow import DeepExplain
from keras import backend as K

Using TensorFlow backend.


In [2]:
with DeepExplain(session=K.get_session()) as de:
    model = Sequential()
    model.add(Dense(2, activation=Activation('relu'), input_dim=2))
    model.add(Dense(2, activation=Activation('relu')))
    model.add(Dense(3, activation=Activation('relu')))
    model.add(Dense(1, activation=Activation('relu')))
    
    w1 = np.array([[1.,0.], [0.,1.]])
    b1 = np.array([0.,0.])
    model.layers[0].set_weights([w1, b1])

    w2 = np.array([[1., 2.],[-1., -1.]])
    b2 = np.array([-1., 0.])
    model.layers[1].set_weights([w2, b2])

    w3 = np.array([[-2., 2., 1.], [3.,1., -2.]])
    b3 = np.array([-1.,0., -2.])
    model.layers[2].set_weights([w3, b3])

    w4 = np.array([[1.],[-1.], [3.]])
    b4 = np.array([1.])
    model.layers[3].set_weights([w4, b4])
    
                                         #                                                 3*alpha]])}))    
    baseline = np.array([[0,0]])
    xs = np.array([[3, 1]])
    input_tensor = model.layers[0].input
    target_tensor = model.layers[-1].output
    #fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    #target_tensor = fModel(input_tensor)
    print(target_tensor.shape)
    print(model.predict(xs)) #sanity check -


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/Users/jsu/opt/anaconda3/lib/python3.7/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


(?, 1)

[[6.]]


In [5]:
model.predict(xs) - model.predict(baseline)

array([[5.],
       [5.],
       [0.]], dtype=float32)

In [3]:
ground_truth = np.array([4.8, -0.2]) # from Exactline 
steps = [1, 5, 20]#, 10, 20, 50, 100]#, 200, 300, 400, 500]
methods = ['idl_true', 'idl']
attribs_2 = {}
diffs_2 = {}
xs = np.array([[2,1],[3, 1], [2, 10]])

with DeepExplain(session=K.get_session()) as de:
    attrib_deeplift = de.explain('deeplift', target_tensor, input_tensor, xs) #1 numpy array
    # print(attrib_deeplift.shape)
    for method in methods:
        attribs_2[method] = []
        diffs_2[method] = []
        for step in steps:
            attrib = de.explain(method, target_tensor, input_tensor, xs, steps=step) #1 numpy array
            attribs_2[method].append(attrib) 
            #diffs_2[method].append(np.sum(np.abs(attrib - ground_truth))) #append 1 number
        
    #attribs['intgrad'] = de.explain('intgrad', target_tensor, input_tensor, xs, steps=steps)
    #attribs['integdeeplift'] = de.explain('integdeeplift', target_tensor, input_tensor, xs, steps=steps)
    print("Done!")
    
print(attribs_2)
print(attrib_deeplift)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(3, 2)
(1, 3, 2)
(3, 2)
Instructions for updating:
Use `tf.variables_initializer` instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
(3, 2)
Done!
{'idl_true': [array([[ 6.66666651, -1.66666663],
       [ 4.84615409,  0.15384603],
       [ 0.        ,  0.        ]]), array([[ 6.66666718, -1.66666679],
       [ 4.8       ,  0.2       ],
       [ 0.        ,  0.        ]]), array([[ 6.66666718, -1.66666679],
       [ 4.8       ,  0.2       ],
       [ 0.        ,  0.        ]])], 'idl': [[array([ 6.66666651, -1.66666663]), array([4.84615409, 0.15384603]), array([0., 0.])], [array([ 6.66666718, -1.66666679]), array([4.8, 0.2]), array([0., 0.])], [array([ 6.66666718, -1.66666679]), array([4.8, 0.2]), array([0., 0.])]]}
[[ 6.6666665  -1.6666666 ]
 [ 4.846154    0.15384603]
 [ 0.          0.        ]]


In [2]:
#toy model with based on Integrated Gradients paper (https://arxiv.org/pdf/1703.01365.pdf)
with DeepExplain(session=K.get_session()) as de:
    model_2 = Sequential()
    model_2.add(Dense(2, activation=Activation('relu'), input_dim=2))
    model_2.add(Dense(1, activation=Activation('relu')))
    
    w1 = np.array([[1, 0], [0, 1]])
    b1 = np.array([-1, 0])
    model_2.layers[0].set_weights([w1, b1]) #x1-1, x2
    
    w2 = np.array([[1], [-1]])
    b2 = np.array([0])
    model_2.layers[1].set_weights([w2, b2]) #z1 - 1 - z2 
    #print(model.layers[0].get_weights())
    
    #alpha = .1
    #print(-15.75*alpha+9)
    #print(K.get_session().run(model.layers[-1].output, feed_dict={model.layers[0].input:np.array([[-2*alpha, 
                                         #                                                 3*alpha]])}))    
    baseline = np.array([[0,0]])
    xs = np.array([[3, 1]])
    input_tensor = model_2.layers[0].input
    target_tensor = model_2.layers[-1].output
    #fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    #target_tensor = fModel(input_tensor)
    print(input_tensor.shape)
    print(target_tensor.shape)
    print(model_2.predict(baseline))
    print(model_2.predict(xs)) #sanity check -
    

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(?, 2)
(?, 1)

[[0.]]
[[1.]]


/Users/jsu/opt/anaconda3/lib/python3.7/site-packages/keras/activations.py:235: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [3]:
ground_truth = np.array([1.5, -0.5]) #self-calculated
steps = [1, 5, 10, 20, 50, 100]#, 1000]# 200, 300, 400, 500]
methods = ['idl_true', 'intgrad']
attribs_2 = {}
diffs_2 = {}
xs = np.array([[3, 1]])

with DeepExplain(session=K.get_session()) as de:
    attrib_deeplift = de.explain('deeplift', target_tensor, input_tensor, xs)[0] #1 numpy array

#     for method in methods:
#         attribs_2[method] = []
#         diffs_2[method] = []
#         for step in steps:
#             attrib = de.explain(method, target_tensor, input_tensor, xs, steps=step)[0] #1 numpy array
#             attribs_2[method].append(attrib) 
#             diffs_2[method].append(np.sum(np.abs(attrib - ground_truth))) #append 1 number
        
    #attribs['intgrad'] = de.explain('intgrad', target_tensor, input_tensor, xs, steps=steps)
    #attribs['integdeeplift'] = de.explain('integdeeplift', target_tensor, input_tensor, xs, steps=steps)
    print("Done!")

#print(attribs_2)
print(attrib_deeplift)

IN GRAD OVERRIDE
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


IN GRAD OVERRIDE
still in init
IN GRAD OVERRIDE
IN GRAD OVERRIDE
Done!
[ 2. -1.]


In [ ]:
import keras
from keras.models import load_model
from keras.models import Model
print("keras version:", keras.__version__)
import tensorflow as tf
print("tensorflow version:", tf.__version__)
import wget
import os
import sys
from keras.datasets import cifar10
import numpy as np
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train, axis=(0, 1, 2, 3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
x_valid = x_train[:10000]
y_valid = y_train[:10000]

model_files = [
    "model_cifar10_balanced_seed-0_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-10_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-20_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-30_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-40_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-50_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-60_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-70_bestbefore-100_currentepoch-100_valacc-91_vgg.h5",
    "model_cifar10_balanced_seed-80_bestbefore-100_currentepoch-100_valacc-90_vgg.h5",
    "model_cifar10_balanced_seed-90_bestbefore-100_currentepoch-100_valacc-91_vgg.h5"
]
model_files = model_files[0] # test with 1 model

for model_file in model_files:
  print("On model", model_file)
#   if (os.path.isfile(model_file)==False):
#     print("Downloading", model_file)
#     wget.download("https://zenodo.org/record/2648107/files/"
#                   +model_file+"?download=1", out=model_file)
  model = load_model(model_file)

  pre_softmax_model = Model(input=model.input,
                            output=model.layers[-2].output)
  print("Making predictions on validation set")
  valid_preacts = pre_softmax_model.predict(x_valid)
  print("Making predictions on test set")
  test_preacts = pre_softmax_model.predict(x_test)

  print("Test accuracy",np.mean(np.argmax(test_preacts,axis=1)
                                == np.squeeze(y_test)))
  print("Valid accuracy",np.mean(np.argmax(valid_preacts,axis=1)
                                 == np.squeeze(y_valid)))


In [ ]:
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    # Need to reconstruct the graph in DeepExplain context, using the same weights.
    # With Keras this is very easy:
    # 1. Get the input tensor to the original model
    input_tensor = model.input # model.layers[0].input
    print(input_tensor)
    print(input_tensor.shape)
    
    # 2. We now target the output of the last dense layer (pre-softmax)
    # To do so, create a new model sharing the same layers untill the last dense (index -2)
    fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
    target_tensor = fModel(input_tensor)
    
    xs = x_test[0:10]
    ys = y_test[0:10]
    
    #attributions_gradin = de.explain('grad*input', target_tensor, input_tensor, xs, ys=ys)
    #attributions_sal   = de.explain('saliency', target_tensor, input_tensor, xs, ys=ys)
    #attributions_ig    = de.explain('intgrad', target_tensor, input_tensor, xs, ys=ys)
    #attributions_dl    = de.explain('deeplift', target_tensor, input_tensor, xs, ys=ys)
    attributions_idl   = de.explain('integdeeplift', target_tensor, input_tensor, xs, ys=ys)